In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from model import NoteComposeNet
from dataset import MidiDataset, VOCABULARY
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt
import librosa 
import torch

In [53]:
df = {
    'notes': [torch.tensor([1 for i in range(0, 255)])],
    'velocities': [torch.tensor([1 for i in range(0, 255)])],
    'durations': [torch.tensor([1 for i in range(0, 255)])],
    'times': [torch.tensor([1 for i in range(0, 255)])],     
}

In [14]:
model = NoteComposeNet()

In [16]:
# Model Specifications
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 0.079MB


In [17]:
midi = MidiDataset(df, context_len = model._context_len)
train_loader = DataLoader(midi, batch_size=1)

data = next(iter(train_loader))
note_tensor = data['notes'].to("cuda")
velocity_tensor = data['velocities'].to("cuda")
duration_tensor = data['durations'].to("cuda")
time_tensor = data['times'].to("cuda")

input = {
    "notes": note_tensor,
    "velocities": velocity_tensor,
    "durations": duration_tensor ,
    "times": time_tensor
}

NameError: name 'df' is not defined

In [19]:
test_inputs = np.random.randint(0, len(VOCABULARY), size=255, dtype=int)
next_notes = model.generate(test_inputs, max_len=100)
print("Note = ", model.detokenize(next_notes))

c:\Users\Joaquin\Desktop\Playground\ML\Composer\model.py:114: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  def detokenize(self, inputs):


Note =  ['A♯-1', 'D♯8', '<EOS>', 'D♯5', 'A1', 'E9', 'A♯0', 'D♯8', 'B2', 'E1', 'A6', 'C♯5', 'G♯3', 'F9', 'C5', 'A6', 'C9', 'B3', 'E1', 'C1', 'C9', 'F5', 'E1', 'A6', 'C9', 'D♯5', 'A1', 'E1', 'D♯5', 'D♯8', 'D♯8', 'B7', 'C♯0', 'F♯9', 'B7', 'F♯1', 'F2', 'D♯5', 'C♯2', 'D7', 'G2', 'A♯0', 'C♯5', 'G5', 'F0', 'D1', 'G♯0', 'E1', 'G♯3', 'E1', 'A1', 'C7', 'F0', 'G0', 'C5', 'F5', 'B3', 'F5', 'E1', 'G2', 'G3', 'A6', 'F2', 'F0', 'B0', 'A6', 'C♯5', 'F♯6', 'C♯2', 'E1', 'A6', 'A1', 'A6', 'A1', 'A6', 'F5', 'G0', 'C5', 'G0', 'A6', 'F5', 'B3', 'D♯-1', 'A6', 'C♯5', 'B3', 'E3', 'A6', 'G0', 'A♯-1', 'F0', 'E0', 'B2', 'E1', 'D2', 'C♯5', 'C♯-1', 'F5', 'E1', 'A6']


# Training

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model import NoteComposeNet
from dataset import MidiDataset
from torch.utils.data import DataLoader

import torch
import pandas as pd

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [3]:
model = NoteComposeNet()

In [4]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=0.001, 
    weight_decay=0.004,
    betas=(0.9, 0.999),
    eps=1e-07, 
    amsgrad=False)

In [5]:
CSV_PATH = r'midi-dataset-mini.csv'
df = pd.read_csv(CSV_PATH)
midi = MidiDataset(df, context_len = model._context_len)
del df

In [6]:
train_loader = DataLoader(
    midi, 
    batch_size=3,
    num_workers=1,
    shuffle=True, 
)

In [48]:
def unpack_batch(batch):
    b, attn, gt = batch
    
    notes = b['notes'].to(model._device)
    notes_gt = gt['notes'].to(model._device)

    output_logits = model.forward(notes)

    return output_logits, notes_gt
    

In [49]:
for i, batch in enumerate(train_loader):
    output_logits, notes_gt = unpack_batch(batch)

    loss = loss_fn(output_logits, notes_gt)

    print(output_logits[0])
    print(notes_gt[0])
    break

tensor([1.0229e-32, 2.8665e-32, 2.3394e-32, 1.8517e-32, 1.3781e-32, 2.9592e-32,
        1.4465e-32, 2.0021e-32, 4.4173e-32, 2.4039e-32, 2.4819e-32, 1.2922e-32,
        3.7706e-32, 1.1136e-32, 2.9599e-32, 2.6426e-32, 3.7953e-32, 2.3808e-32,
        9.8274e-33, 3.3968e-32, 2.4353e-32, 1.1287e-32, 2.2586e-32, 8.3622e-33,
        1.0417e-32, 2.3354e-32, 1.7328e-32, 4.7355e-32, 5.3448e-32, 2.7813e-32,
        9.7023e-33, 3.6294e-32, 1.8851e-32, 3.2778e-32, 4.3769e-32, 5.6649e-32,
        2.6334e-32, 2.0862e-32, 2.4723e-32, 3.8891e-32, 2.8853e-32, 1.2458e-32,
        2.2081e-32, 3.3754e-32, 9.7927e-33, 8.5685e-33, 3.1325e-32, 3.9987e-32,
        5.5781e-32, 1.8470e-32, 1.8923e-32, 4.3796e-32, 2.3029e-32, 1.7737e-32,
        1.3473e-32, 6.0103e-32, 3.1800e-32, 8.2457e-33, 3.4879e-32, 5.1131e-32,
        1.6355e-32, 2.3355e-32, 1.6317e-32, 1.0608e-30, 1.4061e-32, 2.1613e-32,
        2.7231e-32, 4.2099e-32, 2.3400e-32, 1.6047e-32, 1.3757e-32, 9.3226e-33,
        2.2264e-32, 1.9266e-32, 3.1340e-

In [22]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()
        
        output_logits, notes_gt = unpack_batch(batch)
        
        loss = loss_fn(output_logits, notes_gt)
        loss.backward() 

        optimizer.step()
        
        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [23]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/composer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)


    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(train_loader):
            voutputs, vgt = unpack_batch(vdata)
            vloss = loss_fn(voutputs, vgt)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
LOSS train 0.0 valid 3.895735263824463
EPOCH 2:
LOSS train 0.0 valid 3.895735263824463
EPOCH 3:


KeyboardInterrupt: 